In [1]:
import pandas as pd
import numpy as np
import transformers 
import accelerate
import torch

# Load and Explore Customer Support Dataset
Load a real-life customer support dataset (e.g., from Kaggle) and perform exploratory data analysis to understand its structure.

In [15]:
# Load and Explore Customer Support Dataset
# Load the dataset
df = pd.read_csv(r'/kaggle/input/customersupportdataset1/sample.csv')

# Display basic information about the dataset
print(df.info())

# Display the first few rows of the dataset
print(df.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   query          10 non-null     object
 1   true_response  10 non-null     object
dtypes: object(2)
memory usage: 292.0+ bytes
None
                                  query  \
0           How do I reset my password?   
1           What is your refund policy?   
2     Can I change my shipping address?   
3          How long does delivery take?   
4  Do you offer international shipping?   

                                       true_response  
0  Click on 'Forgot Password' on the login page t...  
1   You can return items within 30 days of purchase.  
2  Yes, you can update it before the item is ship...  
3        Delivery typically takes 3-5 business days.  
4          Yes, we ship to most countries worldwide.  


# Generate Structured Output (JSON Mode)
Configure the language model to return responses in a structured JSON format for better control and integration.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
import json

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
hf_token= "104102959865757583751011167382101100107671147079108117856766112120671028489897676107667680"
tokenizer = AutoTokenizer.from_pretrained(model_name,use_auth_token=hf_token)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", use_auth_token=hf_token)

def generate_json_response(query):
    prompt = f"""You are a helpful assistant. Respond to the following query in strict JSON format with keys 'response' and 'confidence'.

Query: "{query}"

Output JSON:"""

    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")
    output = model.generate(**inputs, max_new_tokens=200, do_sample=True, temperature=0.7)
    result = tokenizer.decode(output[0], skip_special_tokens=True)

    
    try:
        json_start = result.index('{')
        json_str = result[json_start:]
        return json.loads(json_str)
    except:
        return {"response": "Could not parse model output", "confidence": 0.0}

query = "What is your refund policy?"
print(generate_json_response(query))


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:897: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/2.10k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/414 [00:00<?, ?B/s]

...

model-00002-of-00002.safetensors:   0%|          | 0.00/4.54G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


{'response': 'Our refund policy varies depending on the product or service. Please refer to our terms and conditions for more information.', 'confidence': 0.8}


# Few-Shot Prompting for Query Understanding
Provide a few examples of customer queries and their expected responses to fine-tune the model's understanding.

In [5]:
few_shot_prompt = """
You are a customer support agent. Here are some examples of queries and responses:

Query: "How can I reset my password?"
Response: "To reset your password, click on 'Forgot Password' on the login page and follow the instructions."

Query: "What is your refund policy?"
Response: "Our refund policy allows returns within 30 days of purchase. Please visit our website for more details."

Now respond to the following query:
Query: "Can I change my shipping address after placing an order?"
Response:
"""

def generate_few_shot_response(query):
    prompt = few_shot_prompt + f'Query: "{query}"\nResponse:'

    
    inputs = tokenizer(prompt, return_tensors="pt").to("cuda")  # Use CPU

    
    output = model.generate(**inputs, max_new_tokens=100, do_sample=True, temperature=0.7)

    
    result = tokenizer.decode(output[0], skip_special_tokens=True)

    return result.strip()


query = "Can I change my shipping address after placing an order?"
print(generate_few_shot_response(query))

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


You are a customer support agent. Here are some examples of queries and responses:

Query: "How can I reset my password?"
Response: "To reset your password, click on 'Forgot Password' on the login page and follow the instructions."

Query: "What is your refund policy?"
Response: "Our refund policy allows returns within 30 days of purchase. Please visit our website for more details."

Now respond to the following query:
Query: "Can I change my shipping address after placing an order?"
Response:
Query: "Can I change my shipping address after placing an order?"
Response: "Yes, you can definitely change your shipping address after placing an order. Please visit our website and log into your account. From there, you can navigate to the 'My Orders' section and select the order you want to update. From there, you can change your shipping address and submit the changes. If you have already received your package, please contact our customer service team and they will assist you with the process

# Embed and Search Customer Queries (Vector Search)
Use embeddings to convert customer queries into vectors and perform similarity searches in the vector database.

In [ ]:
from transformers import AutoTokenizer, AutoModel
import torch
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

model_name = "mistralai/Mistral-7B-Instruct-v0.1"
hf_token = "104102959865757583751011167382101100107671147079108117856766112120671028489897676107667680"

tokenizer = AutoTokenizer.from_pretrained(model_name, use_auth_token=hf_token)
model = AutoModel.from_pretrained(model_name, torch_dtype=torch.float16, device_map="auto", use_auth_token=hf_token)

if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

def get_embedding(text):
    inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True, max_length=512)
    inputs = {k: v.to("cuda") for k, v in inputs.items()}
    with torch.no_grad():
        output = model(**inputs)
        
        embeddings = output.last_hidden_state.mean(dim=1)
    return embeddings.cpu().numpy()

def embed_and_search(query, query_vector_db):
    query_vector = get_embedding(query)
    similarities = cosine_similarity(query_vector, query_vector_db)
    top_k_idx = np.argsort(similarities[0])[::-1][:5]
    return top_k_idx

query_texts = [
    "How can I reset my password?",
    "What is your refund policy?",
    "Can I change my shipping address after placing an order?"
]
query_vector_db = np.vstack([get_embedding(q) for q in query_texts])

query = "How do I change my shipping address?"
search_results = embed_and_search(query, query_vector_db)

print("Top 5 most similar queries:", [query_texts[i] for i in search_results])


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:897: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Top 5 most similar queries: ['How can I reset my password?', 'Can I change my shipping address after placing an order?', 'What is your refund policy?']


# Evaluate the Agent's Performance
Evaluate the agent's responses using metrics like accuracy, relevance, and customer satisfaction scores.

In [11]:
from difflib import SequenceMatcher

def evaluate_responses(true_responses, generated_responses, threshold=0.8, verbose=True):
    assert len(true_responses) == len(generated_responses), "Mismatched list lengths"

    def similarity(a, b):
        return SequenceMatcher(None, a.lower(), b.lower()).ratio()

    correct = 0
    for i, (true, gen) in enumerate(zip(true_responses, generated_responses)):
        sim = similarity(true, gen)
        if sim >= threshold:
            correct += 1
        elif verbose:
            print(f"\n❌ Mismatch at index {i}:")
            print(f"Expected: {true}")
            print(f"Generated: {gen}")
            print(f"Similarity: {sim:.2f}")

    accuracy = correct / len(true_responses)
    print(f"\n✅ Approximate Accuracy (threshold={threshold}): {accuracy * 100:.2f}%")
    return accuracy


# Evaluate the Agent's Response Generation
Now we will create a function that will take input from a compatible csv file and will generate the responses

In [17]:
def run_tests_from_csv(
    csv_path,
    response_model,
    response_tokenizer,
    embedding_model,
    embedding_tokenizer
):
    import pandas as pd
    import numpy as np

    df = pd.read_csv(csv_path)
    queries = df["query"].tolist()
    true_responses = df["true_response"].tolist()


    def generate_json_response(query):
        prompt = f"""You are a helpful assistant. Respond to the following query in strict JSON format with keys 'response' and 'confidence'.

Query: "{query}"

Output JSON:"""

        inputs = response_tokenizer(prompt, return_tensors="pt").to("cuda")
        output = response_model.generate(**inputs, max_new_tokens=200, do_sample=True, temperature=0.7)
        result = response_tokenizer.decode(output[0], skip_special_tokens=True)

        try:
            json_start = result.index('{')
            json_str = result[json_start:]
            return json.loads(json_str)
        except:
            return {"response": "Could not parse model output", "confidence": 0.0}

 
    def get_embedding(text):
        inputs = embedding_tokenizer(text, return_tensors="pt", padding=True, truncation=True).to("cuda")
        with torch.no_grad():
            outputs = embedding_model(**inputs)
            if hasattr(outputs, "last_hidden_state"):
                embeddings = outputs.last_hidden_state.mean(dim=1)
            elif hasattr(outputs, "hidden_states"):
                embeddings = outputs.hidden_states[-1].mean(dim=1)
            else:
                raise ValueError("Model output does not contain embeddings.")
        return embeddings.cpu().numpy()

 
    def embed_and_search(query, query_vector_db):
        query_vector = get_embedding(query)
        similarities = cosine_similarity(query_vector, query_vector_db)
        top_k_idx = np.argsort(similarities[0])[::-1][:5]
        return top_k_idx

    def evaluate_responses(true_responses, generated_responses):
        accuracy = sum([1 for true, gen in zip(true_responses, generated_responses) if true.strip().lower() == gen.strip().lower()]) / len(true_responses)
        print(f"Accuracy: {accuracy * 100:.2f}%")
        return accuracy

    # Run tests
    print("🔍 Testing `generate_json_response`")
    generated_json_responses = []
    for q in queries:
        json_resp = generate_json_response(q)
        generated_json_responses.append(json_resp["response"])
        print(f"Query: {q}\nGenerated: {json_resp}\n")

    print("\n🔍 Testing `embed_and_search`")
    query_vectors = np.array([get_embedding(q) for q in queries]).squeeze(axis=1)
    search_query = "Can I update my shipping details?"
    top_results = embed_and_search(search_query, query_vectors)
    print(f"Top match indexes for '{search_query}': {top_results}")

    print("\n🔍 Testing `evaluate_responses`")
    evaluate_responses(true_responses, generated_json_responses)


# Test Cases
Here are some example testcase of the functions that are taking input from a csv file: sample.csv

In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModel
import torch


response_tokenizer = AutoTokenizer.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", use_auth_token=hf_token)
response_model = AutoModelForCausalLM.from_pretrained("mistralai/Mistral-7B-Instruct-v0.1", torch_dtype=torch.float16, device_map="auto", use_auth_token=hf_token)

if response_tokenizer.pad_token is None:
    response_tokenizer.pad_token = response_tokenizer.eos_token


embedding_tokenizer = response_tokenizer 
embedding_model = response_model 

run_tests_from_csv(
    csv_path="/kaggle/input/customersupportdataset1/sample.csv",
    response_model=response_model,
    response_tokenizer=response_tokenizer,
    embedding_model=embedding_model,
    embedding_tokenizer=embedding_tokenizer
)


/usr/local/lib/python3.11/dist-packages/transformers/models/auto/tokenization_auto.py:897: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/models/auto/auto_factory.py:476: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]